In [1]:
## Query real time data from DingXiangYuan, and keep the latest records every day for each city

## Created on Sat Feb  8 12:41:50 2020
## Author: leebond
#### resource: https://github.com/jianxu305/nCov2019_analysis

import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
from googletrans import Translator # package used to translate Chinese into English
translator = Translator()

In [3]:
## Query the latest data
! python dataset.py

2020-02-10 21:28:03Update records successfully to ../data/DXY_Chinese.csv


In [36]:
DXYArea = pd.read_csv('../data/DXY_Chinese.csv') # Read Chinese version 
# select column
DXYArea = DXYArea[['date','country','countryCode','province', 'city', 'confirmed', 'suspected', 'cured', 'dead']]

DXYArea.tail(2)  

date country countryCode province city  confirmed  suspected  \
8369  2020-02-10      美国          US      NaN  NaN         12          0   
8370  2020-02-10      越南          VN      NaN  NaN         14          0   

      cured  dead  
8369      3     0  
8370      3     0

In [37]:
def isNaN(num):
    return num != num

## Resoruce for Chinese - English Translation
with open('chineseProvince_to_EN.pkl','rb') as f:
    prov_dict = pkl.load(f)

    
with open('chineseCity_to_EN.pkl','rb') as f:
    city_dict = pkl.load(f)    
        
def translate_to_English(data, prov_dict, city_dict):
    """Translate Chinese in dataset to English
    """        
    data['province'] = data['province'].apply(getProvinceTranslation)
    data['city'] = data['city'].apply(getCityTranslation)
    
    for city in unable_translation: # remove these unable translated data
        data = data[data['city']!=city]
    return data
    
def getProvinceTranslation(name):
    if not isNaN(name):
        return prov_dict[name]
    else: 
        return name

unable_translation = []
def getCityTranslation(name):
    try:
        if not isNaN(name): 
            return city_dict[name]
        else:
            return name
    except:
        unable_translation.append(name)
        #print(name + ' cannot be translated\n')
        return name
    

In [38]:
daily_frm_DXYArea = translate_to_English(DXYArea, prov_dict, city_dict)

daily_frm_DXYArea.head(3)

date country countryCode        province   city  confirmed  \
0  2019-12-01      中国          CN             NaN    NaN          1   
1  2019-12-01      中国          CN  Hubei Province    NaN          1   
2  2019-12-01      中国          CN  Hubei Province  Wuhan          1   

   suspected  cured  dead  
0          0      0     0  
1          0      0     0  
2          0      0     0

In [28]:
len(daily_frm_DXYArea) == len(DXYArea)

True

In [39]:
daily_frm_DXYArea.to_csv ('../data/DXYArea.csv', index = None, header=True)